In [1]:
import copy

# 外皮情報LV2からLV3へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）
- 住戸全体の外皮（床下空間に属するものを除く）、および、室と床下空間の間の床は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。
- 各部位の熱橋は、対応する一般部のU値の低減に置き換える。一般部の断熱材の厚を調整する。
- その他は、用途別情報が得られている場合と同様に求める。

### 《土間床等面積が0㎡より大きい場合の扱い》
$$ \qquad
A_{b}=min(A_主 + A_他 + A_非, \sum_{i}^{}S_{df,i})
\qquad
\\
$$

- 入力値：
  - 室用途不明の空間に属する土間床等の部位iの面積$[m ^ 2]$：$S_{df,i}$

- 出力値：

  - 室と床下空間の間の床面積の合計$[m ^ 2]$：$A_{b}$

### 1) 室用途別面積比率

#### Outline

$$ 
r_m = \frac{ a_m }{ A_m + A_o + A_n }
$$

$$
r_o = \frac{ A_o }{ A_m + A_o + A_n }
$$

$$
r_n = \frac{ A_n }{ A_m + A_o + A_n }
$$

$r_m$: 主たる居室の面積比率, -  
$r_o$: その他の居室の面積比率, -  
$r_n$: 非居室の面積比率, -  
$A_m$: 主たる居室の面積, m<sup>2</sup>  
$A_o$: その他の居室の面積, m<sup>2</sup>  
$A_n$: 非居室の面積, m<sup>2</sup>  

#### Function

In [2]:
def get_area_rate(a_m, a_o, a_t):
    # a_m is the main occupied floor area as float, m2
    # a_o is the other occupied floor area as float, m2
    # a_t is the total occupied floor area as float, m2
    
    # a_n is the non ocupied floor area as float, m2
    a_n = a_t - a_m - a_o

    return a_m / a_t, a_o / a_t, a_n / a_t

### 2) 室用途別の面積の推定

#### Outline

$$
A_{m,i}= A_{i} \times r_m
$$

$$
A_{o,i}= A_{i} \times r_o
$$

$$
A_{n,i}= A_{i} \times r_n
$$

$A_{m,i}$: 主たる居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{o,i}$: その他の居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{n,i}$: 非居室の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_i$: 外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>

#### Function

In [3]:
def get_area_by_room_use( target_area, a_m, a_o, a_t ):
    # target_area is the area of the part i as double, m2
    # a_m is the main floor area as float, m2
    # a_o is the other floor area as float, m2
    # a_t is the total floor area as float, m2
    
    r_m, r_o, r_n = get_area_rate( a_m, a_o, a_t )
    return {
        'main'      : r_m * target_area,
        'other'     : r_o * target_area,
        'nonliving' : r_n * target_area
    }

### 3) 室用途別の長さの推定

#### Outline

$$
L_{m,i}= L_{i} \times r_m
$$

$$
L_{o,i}= L_{i} \times r_o
$$

$$
L_{n,i}= L_{i} \times r_n
$$

$L_{m,i}$: 主たる居室の熱橋及び土間床等の外周部$i$の長さ, m  
$L_{o,i}$: その他の熱橋及び土間床等の外周部$i$の長さ, m  
$L_{n,i}$: 非居室の熱橋及び土間床等の外周部$i$の長さ, m  
$L_i$: 熱橋及び土間床等の外周部$i$の長さ, m

#### Function

In [4]:
def get_length_by_room_use(target_length, a_m, a_o, a_t):
    # target_length is the length of the part i as double, m
    # a_m is the main floor area as float, m
    # a_o is the other floor area as float, m
    # a_t is the total floor area as float, m
    
    r_m, r_o, r_n = get_area_rate(a_m, a_o, a_t)
    return {
        'main'      : r_m * target_length,
        'other'     : r_o * target_length,
        'nonliving' : r_n * target_length
    }

#### Example

In [5]:
areas = get_area_by_room_use(50.0, 30.0, 60.0, 120.0)
lengths = get_length_by_room_use(12.0, 30.0, 60.0, 120.0)
areas, lengths

({'main': 12.5, 'other': 25.0, 'nonliving': 12.5},
 {'main': 3.0, 'other': 6.0, 'nonliving': 3.0})

## 2. Convert

### 1) 共通

'Common'要素はそのまま維持する。

#### Input

参照：InputDataDefinition.ipynb

#### Output

参照：InputDataDefinition.ipynb

#### Function

In [6]:
def convert_common(d):
    return d['Common']

#### Example

In [7]:
d = {
    'Common': {
        'Region' : 6,
        'IsSimplifiedInput': True,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    }
}

convert_common(d)

{'Region': 6,
 'IsSimplifiedInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'TotalFloorArea': 120.0}

### 2) 開口部以外の不透明部位

- 壁体名称、一般部位構造種別、壁体種別、方位、層構成（部材名称、厚さ[m]、熱伝導率[W/mK]、容積比熱[(J/(m<sup>3</sup>・K)]）などの情報は維持する。
- Lv2の入力情報に「壁体が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [8]:
def convert_wall( d ):
    
    walls =[]
    for arg_wall in d['Walls']:
        areas = get_area_by_room_use(arg_wall['area'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, area in areas.items() :
             # 壁体名称、部位面積を更新し、「壁体が属する空間種類」を追加する
            wall = copy.deepcopy(arg_wall)
            wall['name'] = arg_wall['name'] + '_' + space 
            wall['area'] = area
            wall['space'] = space
            walls.append(wall)
            
    return walls

#### Example

In [9]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Walls': [
        { 'area': 67.8, 'direction': 'top', 'name': 'Ceiling' },
        { 'area': 40.63, 'direction': 'SW', 'name': 'Wall_SW',}
    ]
}    

convert_wall(d)

[{'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main'},
 {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other'},
 {'area': 16.95,
  'direction': 'top',
  'name': 'Ceiling_nonliving',
  'space': 'nonliving'},
 {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main'},
 {'area': 20.315,
  'direction': 'SW',
  'name': 'Wall_SW_other',
  'space': 'other'},
 {'area': 10.1575,
  'direction': 'SW',
  'name': 'Wall_SW_nonliving',
  'space': 'nonliving'}]

### 3) 開口部の透明部位

- 開口部名称、方位、日射熱取得率[-]、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [10]:
def convert_window( d ):
    
    windows =[]
    for arg_window in d['Windows']:
        areas = get_area_by_room_use(arg_window['area'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, area in areas.items() :
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            window = copy.deepcopy(arg_window)
            window['name'] = arg_window['name'] + '_' + space
            window['area'] = area
            window['space'] = space
            windows.append(window)
        
    return windows

#### Example

In [11]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17  }
    ]
}    

convert_window(d)

[{'name': 'WindowSW_main', 'direction': 'SW', 'area': 7.5625, 'space': 'main'},
 {'name': 'WindowSW_other',
  'direction': 'SW',
  'area': 15.125,
  'space': 'other'},
 {'name': 'WindowSW_nonliving',
  'direction': 'SW',
  'area': 7.5625,
  'space': 'nonliving'},
 {'name': 'WindowNW_main', 'direction': 'NW', 'area': 0.7925, 'space': 'main'},
 {'name': 'WindowNW_other',
  'direction': 'NW',
  'area': 1.585,
  'space': 'other'},
 {'name': 'WindowNW_nonliving',
  'direction': 'NW',
  'area': 0.7925,
  'space': 'nonliving'}]

### 4) 開口部の不透明部位

- 開口部名称、方位、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Function

In [12]:
def convert_door( d ):
    
    doors =[]
    for arg_door in d['Doors']:
        areas = get_area_by_room_use(arg_door['area'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, area in areas.items() :
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            door = copy.deepcopy(arg_door)
            door['name'] = arg_door['name'] + '_' + space
            door['area'] = area
            door['space'] = space
            doors.append(door)
        
    return doors

#### Example

In [13]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 52 }
    ]
}    
convert_door(d)

[{'name': 'DoorNW_main', 'direction': 'NW', 'area': 13.0, 'space': 'main'},
 {'name': 'DoorNW_other', 'direction': 'NW', 'area': 26.0, 'space': 'other'},
 {'name': 'DoorNW_nonliving',
  'direction': 'NW',
  'area': 13.0,
  'space': 'nonliving'}]

### 5) 線熱橋

- 熱橋部の構造種別、方位、線熱貫流率[W/(m・K)]などの情報は維持する。
- 「 熱橋部が属する空間種類」の情報を追加し、熱橋長さから室用途別の熱橋長さを推定する。

#### Function

In [14]:
def convert_heatbridge( d ):
    
    heatbridges =[]
    for arg_heatbridge in d['Heatbridges']:
        lengths = get_length_by_room_use(arg_heatbridge['length'], d['Common']['MainOccupantRoomFloorArea'], d['Common']['OtherOccupantRoomFloorArea'], d['Common']['TotalFloorArea'])
        for space, length in lengths.items() :
            # 熱橋部長さを更新し、「熱橋部が属する空間種類」を追加する
            heatbridge = copy.deepcopy(arg_heatbridge)
            heatbridge['length'] = length
            heatbridge['space'] = space
            heatbridges.append(heatbridge)
        
    return heatbridges

#### Example

In [15]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Heatbridges': [
        { 'structure': 'rc', 'direction1': 'NW', 'direction2': 'NE', 'length': 10 }
    ]
}    
convert_heatbridge(d)

[{'structure': 'rc',
  'direction1': 'NW',
  'direction2': 'NE',
  'length': 2.5,
  'space': 'main'},
 {'structure': 'rc',
  'direction1': 'NW',
  'direction2': 'NE',
  'length': 5.0,
  'space': 'other'},
 {'structure': 'rc',
  'direction1': 'NW',
  'direction2': 'NE',
  'length': 2.5,
  'space': 'nonliving'}]

### 6) 土間床等の外周部の線熱橋

- 方位、熱橋長さ[m]、線熱貫流率[W/(m・K)]などの情報は維持する。
- 「土間床等の外周部の線熱橋が属する空間種類」の情報を追加する。ただし、Lv2からLv3への変換においては、空間種類は「床下空間」のみとする。

#### Function

In [16]:
def convert_earthfloorperimeter( d ):
    
    def make_earthfloorperimeter( f ):
        earthfloor_perimeter = copy.deepcopy( f )
        earthfloor_perimeter['space'] = 'underfloor'
        return earthfloor_perimeter
    
    return [ make_earthfloorperimeter(f) for f in d['EarthfloorPerimeters'] ]

#### Example

In [17]:
d = {
    'EarthfloorPerimeters': [
        { 'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8 },
        { 'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8 }
    ]
}    
convert_earthfloorperimeter(d)

[{'direction': 'NW',
  'length': 2.43,
  'name': 'other_NW',
  'psi': 1.8,
  'space': 'underfloor'},
 {'direction': 'NE',
  'length': 2.43,
  'name': 'other_NE',
  'psi': 1.8,
  'space': 'underfloor'}]

### 7) 土間床
- 面積[m<sup>2</sup>]などの情報は維持する。
- 「土間床等が属する空間種類」の情報を追加する。ただし、Lv2からLv3への変換においては、空間種類は「床下空間」のみとする。

#### Function

In [18]:
def convert_earthfloor( d ):
    
    def make_earthfloor( f ):
        earthfloor = copy.deepcopy( f )
        earthfloor['space'] = 'underfloor'
        return earthfloor
    
    return [ make_earthfloor(f) for f in d['Earthfloors'] ]

#### Example

In [19]:
d = {
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0 }
    ]
}    
convert_earthfloor(d)

[{'name': 'other', 'area': 5.0, 'space': 'underfloor'}]

## 3. Lv3入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床の各コンバート関数を統合し、Lv2の入力情報をLv3の入力情報にコンバートする。

### Function

In [20]:
def convert(d):
    d_lv3_input = { 
        'Common': convert_common(d),
        'Walls' : convert_wall(d),
        'Windows' : convert_window(d),
        'Doors' : convert_door(d),
        'EarthfloorPerimeters' : convert_earthfloorperimeter(d),
        'Earthfloors' : convert_earthfloor(d)
    }
    if 'Heatbridges' in d == True:
        d_lv3_input['Heatbridges'] = convert_heatbridge(d) 
            
    return d_lv3_input

### Example

In [21]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'area': 67.8, 'direction': 'top', 'name': 'Ceiling', 'structure': 'wood', 'type': 'ceiling' },
        { 'area': 40.63, 'direction': 'SW', 'name': 'Wall_SW', 'structure': 'wood', 'type': 'wall' },
    ],
    'Windows': [
        { 'area': 30.25, 'direction': 'SW', 'name': 'WindowSW', 'type': 'single' },
        { 'area': 3.17, 'direction': 'NW', 'name': 'WindowNW', 'type': 'single' },
    ],
    'Doors': [
        { 'area': 2.52, 'direction': 'NW', 'name': 'DoorNW'},
        { 'area': 2.16, 'direction': 'NE', 'name': 'DoorNE'}
    ],
    'EarthfloorPerimeters': [
        { 'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8 },
        { 'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8 }
    ],
    'Earthfloors': [
        { 'area': 5.0, 'name': 'other' },
        { 'area': 5.0, 'name': 'entrance' }
    ]
}

convert(d)

{'Common': {'Region': 6,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'TotalFloorArea': 120.0},
 'Walls': [{'area': 16.95,
   'direction': 'top',
   'name': 'Ceiling_main',
   'structure': 'wood',
   'type': 'ceiling',
   'space': 'main'},
  {'area': 33.9,
   'direction': 'top',
   'name': 'Ceiling_other',
   'structure': 'wood',
   'type': 'ceiling',
   'space': 'other'},
  {'area': 16.95,
   'direction': 'top',
   'name': 'Ceiling_nonliving',
   'structure': 'wood',
   'type': 'ceiling',
   'space': 'nonliving'},
  {'area': 10.1575,
   'direction': 'SW',
   'name': 'Wall_SW_main',
   'structure': 'wood',
   'type': 'wall',
   'space': 'main'},
  {'area': 20.315,
   'direction': 'SW',
   'name': 'Wall_SW_other',
   'structure': 'wood',
   'type': 'wall',
   'space': 'other'},
  {'area': 10.1575,
   'direction': 'SW',
   'name': 'Wall_SW_nonliving',
   'structure': 'wood',
   'type': 'wall',
   'space': 'nonliving'}],
 'Windows': [{'area': 7.5625,
   'di